# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

**Открываю таблицу и изучаю общую информацию о данных методом .info().**   

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
display(data.head(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведу на экран первые 15 строк таблицы методом .head(15).    
Обнаруживаю, что в 2-х столбцах 'days_employed' и 'total_income' есть пропуски.

### Шаг 2.1 Заполнение пропусков

**Узнав, что в 2-х столбцах 'days_employed' и 'total_income' есть пропуски, методом isna() найду все строки с пропусками и просмотрю первые пять. Обнаруживаю, что это значение NaN.**    

**Затем определю, какую долю в процентах составляют пропущенные значения в каждом из столбцов с пропусками. Доля тоже одинаковая.**

Полагаю, что возможными причинами пропусков являются: человеческий фактор, ошибка ввода данных, сокрытие информации.

In [2]:
print('Доля пропусков составляет', (len(data[data['days_employed'].isna()]) / 
      data['days_employed'].value_counts().sum()*100).round(), '%') #определяю долю пропусков в days_employed

Доля пропусков составляет 11.0 %


**Однако, прежде, чем заполнять пропуски NaN, необходимо разобраться с аномалиями, которые я обнаружил.**  
Хотя этот шаг предлагается выполнить позже ("Шаг 2.2"), на мой взгляд, важно и нужно перед заполнением пропусков сначала разобраться с артефактами в столбце days_employed.

**Аномалия №1:** столбец 'days_employed' содержит отрицательные значения, если значение столбца 'income_type' != 'пенсионер'.  
Исправлю это: приму все значения столбца  по модулю (т.к. кол-во дней по смыслу не может быть отрицательным). 

In [3]:
data['days_employed'] = abs(data['days_employed']) #все значения в days_employed сделаю положительными

**Аномалия №2:** столбец 'days_employed' у "пенсионеров" и "безработных" содержит значения в часах, а не днях.  

Для решения этой проблемы сначала проанализирую данные: сформирую и выведу таблицу значений max, min, count, mean и median в столбце 'days_employed'.

In [4]:
parameters_table = data.groupby('income_type').agg({'days_employed': ['mean', 'median', 'min', 'max', 'count']})
display(parameters_table.round()) #сформирую таблицу и проанализирую значения

days_employed                                     
                         mean    median       min       max  count
income_type                                                       
безработный          366414.0  366414.0  337524.0  395303.0      2
в декрете              3297.0    3297.0    3297.0    3297.0      1
госслужащий            3400.0    2689.0      40.0   15193.0   1312
компаньон              2112.0    1547.0      30.0   17616.0   4577
пенсионер            365003.0  365213.0  328729.0  401755.0   3443
предприниматель         521.0     521.0     521.0     521.0      1
сотрудник              2326.0    1574.0      24.0   18389.0  10014
студент                 579.0     579.0     579.0     579.0      1

**Напишу функцию**, которая будет делить значение в столбце 'days_employed' на 24, если значение в столбце 'days_employed' > 18389, поскольку это максимальное значение в столбце (без пенсионеров и безработных).

In [5]:
def hours_to_days(hours):  
    if hours > 18389:      
        days = hours / 24  
    else:
        days = hours
    return (days)

data['days_employed'] = data['days_employed'].apply(hours_to_days)

In [6]:
display(data.head(15).round()) #проверим первые 15 строк изменненого датасета
parameters_table = data.groupby('income_type').agg({'days_employed': ['mean', 'median', 'min', 'max', 'count']})
display(parameters_table.round()) #сформирую таблицу и проанализирую изменённые значения

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438.0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876.0,покупка жилья
1,1,4025.0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.0,приобретение автомобиля
2,0,5623.0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886.0,покупка жилья
3,3,4125.0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629.0,дополнительное образование
4,0,14178.0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.0,сыграть свадьбу
5,0,926.0,27,высшее,0,гражданский брак,1,M,компаньон,0,255764.0,покупка жилья
6,0,2879.0,43,высшее,0,женат / замужем,0,F,компаньон,0,240526.0,операции с жильем
7,0,153.0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135824.0,образование
8,2,6930.0,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95857.0,на проведение свадьбы
9,0,2189.0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144426.0,покупка жилья для семьи


days_employed                                  
                         mean   median      min      max  count
income_type                                                    
безработный           15267.0  15267.0  14064.0  16471.0      2
в декрете              3297.0   3297.0   3297.0   3297.0      1
госслужащий            3400.0   2689.0     40.0  15193.0   1312
компаньон              2112.0   1547.0     30.0  17616.0   4577
пенсионер             15208.0  15217.0  13697.0  16740.0   3443
предприниматель         521.0    521.0    521.0    521.0      1
сотрудник              2326.0   1574.0     24.0  18389.0  10014
студент                 579.0    579.0    579.0    579.0      1

**Наконец, после этого заполню пропуски NaN медианным значением каждой группы в 'income_type'** методом .transform(), поскольку перебирать каждую категорию вручную с помощью цикла и заполняя пропуски - это неоптимальное решение.

In [7]:
data.groupby(['income_type'])['days_employed'].transform('median')
data['days_employed'] = data['days_employed'].fillna(data.groupby(['income_type'])['days_employed'].transform('median'))

In [8]:
display(data[data['days_employed'].isna()]) #проверим отсутствие пропусков в 'days_employed'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


**Теперь заполню пропуски NaN в 'total_income' медианным значением каждой группы аналогично:**

In [9]:
data.groupby(['income_type'])['total_income'].transform('median')
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type'])['total_income'].transform('median'))

In [10]:
display(data[data['total_income'].isna()]) #проверим отсутствие пропусков в 'total_income'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Заполнять пропуски медианным значением — лучшее решение для количественных переменных, поскольку среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства.

**Аномалия №3:** столбцы 'education' и 'family_status' включают в себя разные варианты написания данных (строчные, заглавные буквы). Исправим это.

In [11]:
display(data.head(20))
data['education'] = data['education'].str.lower() #преобразуем в нижний регистр
data['family_status'] = data['family_status'].str.lower() #преобразуем в нижний регистр

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Аномалия №4:**  
Методом unique() выведу уникальные значения столбца 'children'.   
Обнаруживаю, что есть 2 странные группы, где количество детей указано как 20 и -1.

In [12]:
display(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Удалить эти строки можно, поскольку их доля несуществена.   
Но я, полагая, что это человеческая или техническая ошибка, решил методом replace заменить 20 на 2, а -1 на 1.
Затем проверю.

In [13]:
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)

In [14]:
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


**Возможными причинами появления таких данных являются:** неверная запись, неверная классифицикация объектов, ошибки при импорте данных, при объединении таблиц.

### Шаг 2.3. Изменение типов данных.

In [15]:
data['total_income'] = data['total_income'].astype('int') #заменим на целочисленный тип
data.info() #проверим замену

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление и обработка дубликатов.

Обработку дубликатов начну с поиска и удаления явных дубликатов методом drop_duplicates() с изменением индексов.

In [16]:
print(data.duplicated().sum()) #поиск явных дубликатов
data = data.drop_duplicates().reset_index(drop=True) #удаление явных дубликатов
#print(data.duplicated().sum()) #проверим

71


**Теперь займусь обработкой неявных дубликатов.**  
Для этого последовательно выведу на экран уникальные значения столбцов 'gender' и 'purpose'.   
Замечаю странную строку со значением 'XNA' в столбце 'gender'. Проанализировав, что она занимает всего 1 строку из всего датасета, я принял решение её удалить. 

In [17]:
#print(data['gender'].value_counts()) #вывожу уникальные значения
data = data.loc[data['gender'] != 'XNA'] #удаляю строку
print(data['gender'].value_counts()) #проверяю

F    14174
M     7279
Name: gender, dtype: int64


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

**Создам два новых датафрейма** со столбцами согласно описанию проекта.   
Затем **избавлюсь в эти словарях от дубликатов** с помощью функции .drop_duplicates(), чтобы в них было только по одному значению связки id -> название категории.  
**Удалю из исходного датафрейма** столбцы 'education' и 'family_status', оставив только их идентификаторы: 'education_id' и 'family_status_id'.

In [18]:
part_data_edu = data[['education_id', 'education']] #создам датафрейм
part_data_edu = part_data_edu.drop_duplicates().reset_index(drop=True) #избавлюсь от дубликатов

part_data_family = data[['family_status_id', 'family_status']] #создам еще один датафрейм
part_data_family = part_data_family.drop_duplicates().reset_index(drop=True)

data.drop('education', axis=1, inplace=True) #удалим столбец 'education'
data.drop('family_status', axis=1, inplace=True) #удалим столбец 'family_status'
#print(data.info()) #проверим кол-во столбцов в исходном датафрейме

### Шаг 2.6. Категоризация дохода.

Предварительно методом shape выведу на экран кол-во строк и столбцов датафрейма:

In [19]:
print(data.shape)

(21453, 10)


На основании диапазонов, указанных в описании проекта, **создам столбец 'total_income_category' с категориями.**   
Для этого воспользуюсь функцией одной строки.   
Затем методом value_counts() выведу на экран кол-во строк нового столбца.     
Всё совпало. 

In [20]:
def total_income_category_def(func):
    income = func['total_income']

    if income <= 30000:
        return 'E'

    if 30000 < income <= 50000:
        return 'D'

    if 50000 < income <= 200000:
        return 'C'

    if 200000 < income <= 1000000:
        return 'B'

    return 'A'
data['total_income_category'] = data.apply(total_income_category_def, axis=1)
print(data['total_income_category'].value_counts()) #проверка
print(data.shape)

C    16015
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64
(21453, 11)


### Шаг 2.7. Категоризация целей кредита.

Создам функцию, которая на основании данных из столбца 'purpose' сформирует новый столбец 'purpose_category', в который войдут указанные в проекте категории.

In [21]:
def purpose_category_func(category):
    categories = category['purpose']
    
    if 'авто' in categories:
        return 'операции с автомобилем'
    
    if 'жил' in categories or 'недвиж' in categories:
        return 'операции с недвижимостью'                  
    
    if 'свадьб' in categories:
        return 'проведение свадьбы'
    
    if 'образовани' in categories:
        return 'получение образования'
    
    return 'нет категорий'

data['purpose_category'] = data.apply(purpose_category_func, axis=1)
display(data.head()) #проверим

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


**Насколько я понял, словари в этом проекте нужны лишь для ознакомления.**    
В 3-м шаге проекта мне понадобятся названия категорий семейного положения, поэтому сейчас я **обратно объединю словари.**

In [22]:
data = data.merge(part_data_edu, on='education_id', how='left')
data = data.merge(part_data_family, on='family_status_id', how='left')

### Ответы на вопросы.

**Сгруппируем количество задолженностей по количеству детей, используя метод pivot_table():**

In [23]:
debt_with_children = data.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])
display(debt_with_children) 

,count,sum,mean
,debt,debt,debt
children,,,
0,14090,1063,0.075444
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


На основании анализа таблицы **можно сделать вывод** о том, что должников с 5-ю детьми среди кредитополучателей нет. Наибольшее число должников - с 4-мя детьми, а также с 2-мя детьми.   
Можно сделать вывод, что количество детей клиента **влияет** на факт погашения кредита в срок, поскольку разница в процентном отношении (между 7,5% и 9,8%) существенна.

**Сгруппируем количество задолженностей по семейному положению, используя метод pivot_table()::**

In [24]:
debt_family_status = data.pivot_table(index=['family_status'], values='debt', aggfunc=['count', 'sum', 'mean'])
display(debt_family_status)

,count,sum,mean
,debt,debt,debt
family_status,,,
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4150,388,0.093494
женат / замужем,12339,931,0.075452
не женат / не замужем,2810,274,0.097509


На основании анализа таблицы **можно сделать вывод** о том, что кредитополучатели с семейным положением "вдовец/вдова" составляют наименьшую долю должников по кредитам, а кредитополучатели с семейным положением "не женат / не замужем" - составляют наибольшую.    

Можно сделать вывод, что семейное положение клиента **влияет** на факт погашения кредита в срок, поскольку разница в процентном отношении (между 6,6% и 9,8%) существенна.

**Сгруппируем количество задолженностей по уровню дохода, используя метод pivot_table():**

In [25]:
debt_total_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
display(debt_total_income) 

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080000
B,5041,356,0.070621
C,16015,1360,0.084920
D,350,21,0.060000
E,22,2,0.090909


На основании анализа таблицы, **можно сделать вывод** о том, что доля должников по кредитам в категории 'D' (с ежемесячным доходом от 30000 до 50000) наименьшая, а доля должников в категории 'Е' (с ежемесячным доходом 30000 и ниже) - наоборот - наибольшая.   

Можно сделать вывод, что уровень дохода клиента **влияет** на факт погашения кредита в срок, поскольку разница в процентном отношении (между 6% и 9,1%) существенна.

**Сгруппируем количество задолженностей по целям кредита, используя метод pivot_table():**

In [26]:
debt_total_income = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
display(debt_total_income)

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10810,782,0.072340
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034


На основании анализа таблицы, **можно сделать вывод** о том, что доля должников с категорией "операции с недвижимостью" ниже, чем в остальных категориях, а наибольшее число должников - среди кредитополучателей на категорию "операции с автомобилем".    

Можно сделать вывод, что цели кредита клиента **влияют** на факт погашения кредита в срок, поскольку разница в процентном отношении (между 7,2% и 9,4%) существенна.

## Общий вывод:

**Подводя итоги исследования, можно сделать следующие выводы.**   
1) Исходный датафрейм содержал пропущенные значения, различные аномалии и дубликаты. Поэтому перед ответом на поставленный вопрос, я предварительно подготовил датайфрем к анализу.    
    
2) Чтобы разобраться, влияет ли семейное положение, количество детей клиента, уровень его дохода и цели кредита на факт погашения кредита в срок, я сформировал по каждому указанному аспекту таблицу. В каждой таблице я посчитал количество должников, общее количество кредитополучателей и процент должников от общего числа кредитополучателей.
Выводы описал после таблицы. 

**Подводя общий итог:** можно сказать, что семейное положение, количество детей клиента, уровень его дохода и цели кредита **влияют** на факт погашения кредита в срок, поскольку разница в процентном отношении существенна, исходя из анализа таблиц.